In [ ]:
# Esse código pula alguns produtos mas extrái, anda pelas páginas e divide entre categorias e marcas.
import requests
from bs4 import BeautifulSoup
import time

# Função para criar a URL com filtro (categoria) e marca
def generate_url(filtro, brand, page_num):
    # Monta a URL com base na categoria (filtro) e marca
    url = f'https://www.drogariasminasmais.com.br/medicamentos/{filtro}/{brand}?initialMap=c&initialQuery=medicamentos&map=category-1,category-2,brand'
    # Adiciona a paginação
    return f'{url}&page={page_num}'

# Função para buscar os produtos de uma categoria e marca específica
def fetch_products_for_category_and_brand(filtro, brand):
    all_products = []  # Lista para armazenar todos os produtos encontrados
    page_num = 1  # Iniciar com a primeira página

    while True:
        # Gerar a URL para a página atual com o filtro e marca
        url = generate_url(filtro, brand, page_num)
        response = requests.get(url)
        
        if response.status_code != 200:
            print(f"Erro ao acessar a página {page_num}: {response.status_code}")
            break
        
        # Usando BeautifulSoup para parsear o conteúdo
        soup = BeautifulSoup(response.content, 'html.parser')

        # Encontrando os elementos dos produtos na página
        product_name_elements = soup.find_all('span', class_='vtex-product-summary-2-x-productBrand')
        price_elements = soup.find_all('span', class_='vtex-product-price-1-x-sellingPriceValue')

        # Se não houver produtos, paramos a execução
        if not product_name_elements:
            print(f'Não foram encontrados produtos na página {page_num}.')
            break
        
        # Exibindo produtos e preços
        for name, price in zip(product_name_elements, price_elements):
            product_name = name.get_text(strip=True)
            price_value = price.get_text(strip=True)
            all_products.append({'name': product_name, 'price': price_value})
            print(f'Produto: {product_name} - Preço: {price_value}')
        
        # Verificando se há mais páginas
        page_num += 1

    return all_products

# Função principal para buscar produtos por categoria (filtro) e todas as marcas
def search_products_for_category(filtro, brands):
    print(f'\nBuscando produtos para a categoria: {filtro}')
    
    all_category_products = []  # Lista para armazenar todos os produtos de uma categoria
    
    # Loop para buscar todos os produtos de cada marca dentro da categoria
    for brand in brands:
        print(f'\nBuscando produtos para a marca: {brand} na categoria: {filtro}')
        products = fetch_products_for_category_and_brand(filtro, brand)
        all_category_products.extend(products)  # Adiciona os produtos dessa marca à lista geral
        print(f'Total de produtos encontrados para a categoria {filtro} - marca {brand}: {len(products)}')
    
    print(f'\nTotal de produtos encontrados para a categoria {filtro}: {len(all_category_products)}')

# Exemplos de filtros (categorias) e marcas
filtros = [
    'mais-medicamentos', 'vitaminas', 'calmantes', 'gripe-e-resfriado', 'analgésicos', 'similar', 'hipertensão', 'anti-inflamatórios', 
    'colesterol-e-triglicerídeos', 'anticoncepcionais', 'alergias e infecções', 'fitoterápicos', 'genéricos', 'dor-e-febre', 'digestivo', 
    'relaxante-muscular', 'diabetes', 'gastrite', 'emagrecedores', 'disfunção-erétil'
]

brands = [
    'ems', 'ache-rx', 'eurofarma-rx', 'ems-generico', 'sigma-pharma-rx', 'eurofarma-generico', 'cimed', 'biolab-rx', 'libbs', 'germed-generico',
    'apsen', 'medley-genericos', 'momenta', 'supera', 'farmoquimica', 'myralis', 'prati-donaduzzi-generico', 'pharlab', 'abbott-rx', 'torrent', 
    'uniao-quimica-genom', 'ache-otc', 'cristalia-latinofarma', 'teuto-generico', 'asepxia', 'acnase', 'sanofi-rx', 'neo-quimica-generico', 'biolab-generico', 
    'gsk-otc', 'novartis-rx', 'zanphy', 'marjan', 'neo-quimica-similar', 'procter-gambl', 'gsk-rx', 'nova-quimica-generico', 'merck-rx', 
    'airela', 'servier', 'allergan', 'uniao-quimica-generico', 'alcon', 'biolab-avert-otc', 'geolab-generico', 'zodiac-adium', 'sankyo-pharma', 
    'cellera', 'teuto-similar', 'divcom', 'bayer-rx', 'astrazeneca', 'legrand-generico', 'herbarium', 'biosintetica-generico', 'sigma-pharma-otc', 
    'megalabs', 'boehringer', 'bayer-otc', 'brace-pharma-rx', 'hertz-p-2', 'takeda', 'globo-generico', 'reckitt-benckiser-rb', 'cifarma', 'ranbaxy',
    'natulab', 'biosintetica', 'aspen', 'zambon', 'arovida', 'arese', 'viatris', 'sandoz-generico', 'pfizer', 'gross', 'ems-similar', 'lundbeck', 'jj-otc', 
    'cristalia', 'belfar', 'massime', 'hertz-p-1', 'geolab', 'galderma', 'cifarma-similar', 'chiesi-farmalab', 'janssen', 'feelclean', 'elseve', 'althaia', 
    'grunenthal', 'baldacci', 'abv', 'uniao-quimica', 'theraskin', 'besins', 'exeltis', 'torrent-generico', 'merck-generico', 'maxinutri', 'm-s-d', 
    'hebron-quesalon', 'gummy', 'delta', 'bausch-lomb', 'vitamedic-generico', 'nikkho-zydus', 'lilly', 'leopharma', 'pharmascience', 'medquimica-generico', 
    'diffucap-chemobras', 'usk-rx', 'multilab', 'moksha8', 'medquimica', 'm-s-d-organon', 'catarinense', 'boiron', 'nativita-generico', 'globo', 'astellas', 
    'unichems', 'hisamitsu', 'health-labs', 'gsk-stiefel', 'farmax', 'eurofarma-oaz', 'abbott-nutricional', 'weleda', 'vitafor', 'ucb-biopharma', 'sandoz-rx', 
    'bago', 'waldemiro', 'valeant', 'osorio-de-moraes', 'mundipharma', 'medley-rx', 'legrand-rx', 'arte-nativa', 'zydus-generico', 'wyeth-rx', 'vitamedic', 
    'united-rx', 'roche', 'natcofarma', 'caninha', 'blau', 'shire', 'prati-donaduzzi', 'glenmark', 'genomma', 'equaliv', 'diversos-similares', 'biolab-similar', 
    'bio-soft', 'almaroni', 'teva', 'philips-avente', 'organon', 'mylan', 'imec', 'herbamed', 'farmace-generico', 'farmabraz-beta-atalaia-ltda', 'eli-lilly', 
    'cifarma-generico', 'casa-granado', 'umuarama', 'tresemm', 'trb', 'semina', 'sanitall', 'pequenin', 'olvebra', 'novartis-otc', 'nova-quimica-similar', 
    'nestle-industrial', 'nativita-similar', 'minancora', 'millet-roux', 'melpoejo', 'melora', 'latinofarma', 'laboratorios-ferring', 'johnson-johnson-s-a', 
    'glenmark-generico', 'gigita', 'curatec', 'cheez-it', 'cazi', 'bunker', 'brasterapica', 'biomm', 'bianco', 'almeida-prado-ltda', 'abl-brasil'

]

# Executar a busca para uma categoria específica
category_to_search = 'mais-medicamentos'  # Escolha a categoria que deseja buscar
search_products_for_category(category_to_search, brands)


In [28]:
import requests
from bs4 import BeautifulSoup
import json
import re

# fazendo a requisição ao site
requisicao = requests.get('https://www.drogariasminasmais.com.br/medicamentos')

# transformando em objeto soup
soup = BeautifulSoup(requisicao.text, 'html.parser')

# encontrando todos os scripts no HTML
scripts = soup.find_all('script')

# procurando o script que contém a variável __STATE__
for script in scripts:
    if '__STATE__' in script.text:
        # Usando expressão regular para capturar o JSON que está sendo atribuído a __STATE__
        match = re.search(r'__STATE__\s*=\s*(\{.*?\});', script.text, re.DOTALL)
        if match:
            # Carregando o conteúdo JSON
            json_file = json.loads(match.group(1))
            break
